In [1]:
star_solo_hydrop_rna_mapping_unique_cellfilter_dropseq () {
    local fastq_R1_filename="${1}";
    local fastq_R2_filename="${2}";
    local bam_filename="${3}";
    star_reference_dir='/lustre1/project/stg_00002/lcb/fderop/data/00000000_genomes/mm10_STAR_2.7.8_premrna';
    sjdbgtf='/lustre1/project/stg_00002/lcb/fderop/data/00000000_genomes/mm10_STAR_2.7.8_premrna/genes.gtf'
    #module load STAR/2.7.5b-foss-2018a;
    #module load SAMtools/1.10-foss-2018a;
    # Map with STAR solo
    /lustre1/project/stg_00002/lcb/fderop/scripts/STAR-2.7.8a/bin/Linux_x86_64/STAR \
        --runThreadN 36 \
        --runMode alignReads \
        --outSAMtype BAM SortedByCoordinate \
        --sysShell /bin/bash \
        --genomeDir "${star_reference_dir}" \
        --readFilesIn "${fastq_R1_filename}" "${fastq_R2_filename}" \
        --readFilesCommand 'gzip -c -d' \
        --soloCBwhitelist None \
        --soloType CB_UMI_Simple \
        --soloCBstart  1 \
        --soloCBlen 12 \
        --soloUMIstart 13 \
        --soloUMIlen 8\
        --sjdbGTFfile $sjdbgtf \
        --soloCellFilter CellRanger2.2 2000 0.99 10 \
        --outFilterMultimapNmax 1 \
        --outSAMattributes NH HI AS nM CB UB CR CY UR UY \
        --outFileNamePrefix ${bam_filename%bam} \
        --outReadsUnmapped Fastx \
        --quantMode GeneCounts \
        --bamRemoveDuplicatesType UniqueIdentical \
        --soloFeatures Gene GeneFull
        # Index BAM file.
    # samtools index "${bam_filename%bam}Aligned.sortedByCoord.out.bam"
}

Now we run this star solo script in a for loop on all the R2 fastq files in `fastq/`

In [2]:
dir=demultiplexed_dropseq_sub
mkdir $dir
for sample in fastq_dropseq_sub/*_2.SUB.fastq.gz
do
    NAME=${sample%_2.SUB.fastq.gz}
    OUT=${NAME##*/}
    echo $NAME
    star_solo_hydrop_rna_mapping_unique_cellfilter_dropseq \
        ${NAME}_2.SUB.fastq.gz \
        ${NAME}_1.SUB.fastq.gz \
        $dir/$OUT.bam
done

fastq_dropseq_sub/SRR1853178
Jan 31 01:30:48 ..... started STAR run
Jan 31 01:30:48 ..... loading genome
Jan 31 01:31:18 ..... processing annotations GTF
Jan 31 01:31:24 ..... started mapping
Jan 31 01:59:17 ..... finished mapping
Jan 31 01:59:21 ..... started Solo counting
Jan 31 02:06:33 ..... finished Solo counting
Jan 31 02:06:33 ..... started sorting BAM
Jan 31 02:11:58 ..... finished successfully
fastq_dropseq_sub/SRR1853179
Jan 31 02:12:18 ..... started STAR run
Jan 31 02:12:18 ..... loading genome
Jan 31 02:12:49 ..... processing annotations GTF
Jan 31 02:12:54 ..... started mapping
Jan 31 02:51:01 ..... finished mapping
Jan 31 02:51:05 ..... started Solo counting
Jan 31 03:00:52 ..... finished Solo counting
Jan 31 03:00:52 ..... started sorting BAM
Jan 31 03:08:14 ..... finished successfully
fastq_dropseq_sub/SRR1853180
Jan 31 03:08:33 ..... started STAR run
Jan 31 03:08:33 ..... loading genome
Jan 31 03:08:48 ..... processing annotations GTF
Jan 31 03:08:50 ..... started mapp

Now, we can get a summary of the statistics below:

In [3]:
for sample in demultiplexed_dropseq/*/Gene/Summary.csv
do
    echo ${sample#*/}
    cat $sample
    cut -d, -f2 $sample > ${sample%/Gene/Summary.csv}/Gene/Summary_numbers.csv
    printf "\n"
done

SRR1853178.Solo.out/Gene/Summary.csv
Number of Reads,510210716
Reads With Valid Barcodes,1
Sequencing Saturation,0.757257
Q30 Bases in CB+UMI,-nan
Q30 Bases in RNA read,0.62094
Reads Mapped to Genome: Unique+Multiple,0.545704
Reads Mapped to Genome: Unique,0.545704
Reads Mapped to Transcriptome: Unique+Multipe Genes,0.372239
Reads Mapped to Transcriptome: Unique Genes,0.362322
Estimated Number of Cells,2629
Reads in Cells Mapped to Unique Genes,54743780
Fraction of Reads in Cells,0.296135
Mean Reads per Cell,20823
Median Reads per Cell,15973
UMIs in Cells,7773057
Mean UMI per Cell,2956
Median UMI per Cell,2202
Mean Genes per Cell,1651
Median Genes per Cell,1391
Total Genes Detected,22048

SRR1853179.Solo.out/Gene/Summary.csv
Number of Reads,524537760
Reads With Valid Barcodes,1
Sequencing Saturation,0.768773
Q30 Bases in CB+UMI,-nan
Q30 Bases in RNA read,0.620952
Reads Mapped to Genome: Unique+Multiple,0.543425
Reads Mapped to Genome: Unique,0.543425
Reads Mapped to Transcriptome: Uniq

In [ ]:
ncells_dict = {
    'SRR1853178': 6600,
    'SRR1853179': 9000,
    'SRR1853180': 6120,
    'SRR1853181': 7650,
    'SRR1853182': 7650,
    'SRR1853183': 8280,
    'SRR1853184': 4000,
}

In [15]:
OUT=SRR1853178
ncells=6600
/lustre1/project/stg_00002/lcb/fderop/scripts/STAR-2.7.8a/bin/Linux_x86_64/STAR \
    --runThreadN 36 \
    --runMode soloCellFiltering demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/raw/ demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/filtered_$ncells/ \
    --soloCellFilter CellRanger2.2 $ncells 0.99 10 \
    --soloCBmatchWLtype 1MM \
    --quantMode GeneCounts \
    --soloFeatures Gene GeneFull

Feb 09 15:27:17 ..... started STAR run
Feb 09 15:27:17 ..... starting SoloCellFiltering
Feb 09 15:27:21 ..... finished successfully


In [16]:
OUT=SRR1853179
ncells=9000
gunzip demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/raw/*

/lustre1/project/stg_00002/lcb/fderop/scripts/STAR-2.7.8a/bin/Linux_x86_64/STAR \
    --runThreadN 36 \
    --runMode soloCellFiltering demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/raw/ demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/filtered_$ncells/ \
    --soloCellFilter CellRanger2.2 $ncells 0.99 10 \
    --soloCBmatchWLtype 1MM \
    --quantMode GeneCounts \
    --soloFeatures Gene GeneFull

gzip: demultiplexed_dropseq_sub_whitelist/SRR1853179.Solo.out/Gene/raw/barcodes.tsv: unknown suffix -- ignored
gzip: demultiplexed_dropseq_sub_whitelist/SRR1853179.Solo.out/Gene/raw/features.tsv: unknown suffix -- ignored
gzip: demultiplexed_dropseq_sub_whitelist/SRR1853179.Solo.out/Gene/raw/matrix.mtx: unknown suffix -- ignored
Feb 09 15:27:21 ..... started STAR run
Feb 09 15:27:21 ..... starting SoloCellFiltering
Feb 09 15:27:26 ..... finished successfully


In [17]:
OUT=SRR1853180
ncells=6120
gunzip demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/raw/*

/lustre1/project/stg_00002/lcb/fderop/scripts/STAR-2.7.8a/bin/Linux_x86_64/STAR \
    --runThreadN 36 \
    --runMode soloCellFiltering demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/raw/ demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/filtered_$ncells/ \
    --soloCellFilter CellRanger2.2 $ncells 0.99 10 \
    --soloCBmatchWLtype 1MM \
    --quantMode GeneCounts \
    --soloFeatures Gene GeneFull

gzip: demultiplexed_dropseq_sub_whitelist/SRR1853180.Solo.out/Gene/raw/barcodes.tsv: unknown suffix -- ignored
gzip: demultiplexed_dropseq_sub_whitelist/SRR1853180.Solo.out/Gene/raw/features.tsv: unknown suffix -- ignored
gzip: demultiplexed_dropseq_sub_whitelist/SRR1853180.Solo.out/Gene/raw/matrix.mtx: unknown suffix -- ignored
Feb 09 15:27:26 ..... started STAR run
Feb 09 15:27:26 ..... starting SoloCellFiltering
Feb 09 15:27:29 ..... finished successfully


In [18]:
OUT=SRR1853181
ncells=7650
gunzip demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/raw/*

/lustre1/project/stg_00002/lcb/fderop/scripts/STAR-2.7.8a/bin/Linux_x86_64/STAR \
    --runThreadN 36 \
    --runMode soloCellFiltering demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/raw/ demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/filtered_$ncells/ \
    --soloCellFilter CellRanger2.2 $ncells 0.99 10 \
    --soloCBmatchWLtype 1MM \
    --quantMode GeneCounts \
    --soloFeatures Gene GeneFull

gzip: demultiplexed_dropseq_sub_whitelist/SRR1853181.Solo.out/Gene/raw/barcodes.tsv: unknown suffix -- ignored
gzip: demultiplexed_dropseq_sub_whitelist/SRR1853181.Solo.out/Gene/raw/features.tsv: unknown suffix -- ignored
gzip: demultiplexed_dropseq_sub_whitelist/SRR1853181.Solo.out/Gene/raw/matrix.mtx: unknown suffix -- ignored
Feb 09 15:27:29 ..... started STAR run
Feb 09 15:27:29 ..... starting SoloCellFiltering
Feb 09 15:27:33 ..... finished successfully


In [19]:
OUT=SRR1853182
ncells=7650
gunzip demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/raw/*

/lustre1/project/stg_00002/lcb/fderop/scripts/STAR-2.7.8a/bin/Linux_x86_64/STAR \
    --runThreadN 36 \
    --runMode soloCellFiltering demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/raw/ demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/filtered_$ncells/ \
    --soloCellFilter CellRanger2.2 $ncells 0.99 10 \
    --soloCBmatchWLtype 1MM \
    --quantMode GeneCounts \
    --soloFeatures Gene GeneFull

gzip: demultiplexed_dropseq_sub_whitelist/SRR1853182.Solo.out/Gene/raw/barcodes.tsv: unknown suffix -- ignored
gzip: demultiplexed_dropseq_sub_whitelist/SRR1853182.Solo.out/Gene/raw/features.tsv: unknown suffix -- ignored
gzip: demultiplexed_dropseq_sub_whitelist/SRR1853182.Solo.out/Gene/raw/matrix.mtx: unknown suffix -- ignored
Feb 09 15:27:34 ..... started STAR run
Feb 09 15:27:34 ..... starting SoloCellFiltering
Feb 09 15:27:38 ..... finished successfully


In [20]:
OUT=SRR1853183
ncells=8280
gunzip demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/raw/*

/lustre1/project/stg_00002/lcb/fderop/scripts/STAR-2.7.8a/bin/Linux_x86_64/STAR \
    --runThreadN 36 \
    --runMode soloCellFiltering demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/raw/ demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/filtered_$ncells/ \
    --soloCellFilter CellRanger2.2 $ncells 0.99 10 \
    --soloCBmatchWLtype 1MM \
    --quantMode GeneCounts \
    --soloFeatures Gene GeneFull

gzip: demultiplexed_dropseq_sub_whitelist/SRR1853183.Solo.out/Gene/raw/barcodes.tsv: unknown suffix -- ignored
gzip: demultiplexed_dropseq_sub_whitelist/SRR1853183.Solo.out/Gene/raw/features.tsv: unknown suffix -- ignored
gzip: demultiplexed_dropseq_sub_whitelist/SRR1853183.Solo.out/Gene/raw/matrix.mtx: unknown suffix -- ignored
Feb 09 15:27:39 ..... started STAR run
Feb 09 15:27:39 ..... starting SoloCellFiltering
Feb 09 15:27:42 ..... finished successfully


In [22]:
OUT=SRR1853184
ncells=4000
gunzip demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/raw/*

/lustre1/project/stg_00002/lcb/fderop/scripts/STAR-2.7.8a/bin/Linux_x86_64/STAR \
    --runThreadN 36 \
    --runMode soloCellFiltering demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/raw/ demultiplexed_dropseq_sub_whitelist/$OUT.Solo.out/Gene/filtered_$ncells/ \
    --soloCellFilter CellRanger2.2 $ncells 0.99 10 \
    --soloCBmatchWLtype 1MM \
    --quantMode GeneCounts \
    --soloFeatures Gene GeneFull

Feb 09 15:28:16 ..... started STAR run
Feb 09 15:28:16 ..... starting SoloCellFiltering
Feb 09 15:28:19 ..... finished successfully


In [25]:
for bcs in demultiplexed_dropseq_sub_whitelist/*.Solo.out/Gene/filtered_*/barcodes.tsv
do
    echo $bcs
    wc -l $bcs
done

demultiplexed_dropseq_sub_whitelist/SRR1853178.Solo.out/Gene/filtered_6600/barcodes.tsv
4871 demultiplexed_dropseq_sub_whitelist/SRR1853178.Solo.out/Gene/filtered_6600/barcodes.tsv
demultiplexed_dropseq_sub_whitelist/SRR1853179.Solo.out/Gene/filtered_9000/barcodes.tsv
5600 demultiplexed_dropseq_sub_whitelist/SRR1853179.Solo.out/Gene/filtered_9000/barcodes.tsv
demultiplexed_dropseq_sub_whitelist/SRR1853180.Solo.out/Gene/filtered_6120/barcodes.tsv
3304 demultiplexed_dropseq_sub_whitelist/SRR1853180.Solo.out/Gene/filtered_6120/barcodes.tsv
demultiplexed_dropseq_sub_whitelist/SRR1853181.Solo.out/Gene/filtered_7650/barcodes.tsv
3578 demultiplexed_dropseq_sub_whitelist/SRR1853181.Solo.out/Gene/filtered_7650/barcodes.tsv
demultiplexed_dropseq_sub_whitelist/SRR1853182.Solo.out/Gene/filtered_7650/barcodes.tsv
3892 demultiplexed_dropseq_sub_whitelist/SRR1853182.Solo.out/Gene/filtered_7650/barcodes.tsv
demultiplexed_dropseq_sub_whitelist/SRR1853183.Solo.out/Gene/filtered_8280/barcodes.tsv
4565 de

4871+5600+3304+3578+3892+4565+3135=28945
Way smaller than 49300.